# Youtube Data Exploration & Cleaning

!["Bronze Button"](https://pbs.twimg.com/media/C1WZpigXUAQMQLj.jpg)

## Imports

In [4]:
import pandas as pd

In [5]:
caVideo_df = pd.read_csv("../Data/CAvideos.csv")

## Inspection

In [51]:
caVideo_columns = list(caVideo_df.columns)

caVideo_nullCount = {}
for col in caVideo_columns:
    null_val = caVideo_df[col].isnull().values.sum()
    caVideo_nullCount[col] = null_val

In [52]:
print(caVideo_nullCount)

{'views': 0, 'comments_disabled': 0, 'video_error_or_removed': 0, 'comment_count': 0, 'channel_title': 0, 'tags': 0, 'dislikes': 0, 'description': 309, 'ratings_disabled': 0, 'trending_date': 0, 'likes': 0, 'video_id': 0, 'category_id': 0, 'publish_time': 0, 'title': 0, 'thumbnail_link': 0}


In [35]:
usaVideo_df = pd.read_csv("../Data/USvideos.csv")

In [54]:
usaVideo_columns = list(usaVideo_df.columns)

In [57]:
usaVideo_nullCount = {}
for col in usaVideo_columns:
    us_null = usaVideo_df[col].isnull().values.sum()
    usaVideo_nullCount[col] = us_null 

In [58]:
usaVideo_nullCount

{'category_id': 0,
 'channel_title': 0,
 'comment_count': 0,
 'comments_disabled': 0,
 'description': 161,
 'dislikes': 0,
 'likes': 0,
 'publish_time': 0,
 'ratings_disabled': 0,
 'tags': 0,
 'thumbnail_link': 0,
 'title': 0,
 'trending_date': 0,
 'video_error_or_removed': 0,
 'video_id': 0,
 'views': 0}

## Cleaning

## Export

## Resources